In [9]:


import matplotlib.pyplot as plt
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import seaborn as sns

dataset = pd.read_csv(r'moods_preprocessed.csv')

dataset.head()



,Unnamed: 0,tempo,instrumentalness,acousticness,energy,valence,loudness,mood
0,0,118.523,0.000000,0.13700,0.730,0.625,0.779912,Happy
1,1,120.255,0.196000,0.01890,0.832,0.166,0.852145,Sad
2,2,136.168,0.000077,0.91300,0.139,0.116,0.615590,Sad
3,3,193.100,0.000006,0.08900,0.438,0.587,0.672509,Happy
4,4,169.881,0.000000,0.00171,0.932,0.445,0.885932,Energetic


In [10]:
dataset['mood']=dataset['mood'].map({'Happy':0, 'Sad':1, 'Energetic': 2, 'Calm':3 }).astype(int)
dataset.head()




,Unnamed: 0,tempo,instrumentalness,acousticness,energy,valence,loudness,mood
0,0,118.523,0.000000,0.13700,0.730,0.625,0.779912,0
1,1,120.255,0.196000,0.01890,0.832,0.166,0.852145,1
2,2,136.168,0.000077,0.91300,0.139,0.116,0.615590,1
3,3,193.100,0.000006,0.08900,0.438,0.587,0.672509,0
4,4,169.881,0.000000,0.00171,0.932,0.445,0.885932,2


In [11]:
data= pd.DataFrame(dataset, columns=['tempo', 'acousticness', 'instrumentalness', 'valence', 'energy', 'loudness', 'mood'])
data.head()
x_data=data.drop(['mood'], axis=1)
y_data=data['mood']
MinMaxScaler= preprocessing.MinMaxScaler()
X_data_minmax= MinMaxScaler.fit_transform(x_data)

X_train, X_test, y_train, y_test = train_test_split(x_data, y_data,test_size=0.2, random_state = 1)

D_train = xgb.DMatrix(X_train, label=y_train)
D_test = xgb.DMatrix(X_test, label=y_test)

param = {
    'eta': 0.1, 
    'max_depth': 2,  
    'objective': 'multi:softprob',  
    'num_class': 4
  
    
} 

steps = 100  # The number of training iterations

model = xgb.train(param, D_train, steps)

import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, best_preds)))


[20:08:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Precision = 0.8127620118899189
Recall = 0.8147219424393338
Accuracy = 0.8333333333333334
